In [ ]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import reconnect_degree_2
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends, solve_degree4, clean_obvious_fake_tips
from extract_graph import prune_graph



In [ ]:
plate = 13
exp_clean = pickle.load(open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )

In [ ]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)
solved,solved_nodes = solve_degree4(exp_clean)
clean_obvious_fake_tips(exp_clean)

In [ ]:
len(exp_clean.hyphaes)

In [135]:
hyphs,gr_inf = save_hyphaes(exp_clean)

4279 4279
exactly on it
5671 5671
exactly on it
5671 5671
exactly on it
5671 5671
exactly on it
11982 11982
exactly on it
11982 11982
exactly on it


In [136]:
exp_clean.save()

In [96]:
plt.close('all')
hyph = choice(exp_clean.hyphaes)
print(hyph.ts)
t0 = choice(hyph.ts)

[20, 21]


In [97]:
exp_clean.plot([t0],[[hyph.root.label,hyph.end.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
clean_obvious_fake_tips(exp_clean)

[(580355.0, 2420)]
[(601137.0, 2523)]
[(623346.0, 2598)]
[(639622.0, 2673)]
[(665367.0, 2763)]
[(694470.0, 2872)]
[(726362.0, 3057)]
[(765948.0, 3204)]
[(799091.0, 3355)]
[(830005.0, 3470)]
[(850856.0, 3522)]
[(874814.0, 3633)]
[(897835.0, 3766)]
[(949645.0, 4037)]
[(975527.0, 4077)]
[(994536.0, 4211)]
[(1020375.0, 4380)]
[(1049896.0, 4466)]
[(1075976.0, 4560)]
[(1083836.0, 4628)]
[(1102880.0, 4675)]
[(1120373.0, 4767)]
[(1149365.0, 5187)]


In [6]:
number_face=[2+len(g.edges)-len(g.nodes) for g in exp_clean.nx_graph]

In [7]:
number_anas_theory = [number_face[i+1]-number_face[i] for i in range(len(number_face)-2)]

In [122]:
for hyph in exp_clean.hyphaes:
    hyph.update_ts()
to_remove = []
for hyph in exp_clean.hyphaes:
    hyph.update_ts()
    if len(hyph.ts)==0:
        to_remove.append(hyph)
for hyph in to_remove:
    exp_clean.hyphaes.remove(hyph)

In [12]:
hyph_anas_tip_hyph = [hyphat for hyphat in exp_clean.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
hyph_anas_tip_tip = []
potential = []
for hyph in exp_clean.hyphaes:
    if len(hyph.ts)>=2 and hyph.end.degree(hyph.ts[-1])==1 and hyph.end.ts()[-1]!=len(exp_clean.nx_graph)-1 and not np.all([hyph.get_length_pixel(t)<=20 for t in hyph.ts]):
        potential.append(hyph)
for hyph in potential:
    t0 = hyph.ts[-1]
    for hyph2 in potential:
        if hyph2.ts[-1] == t0 and hyph!=hyph2:
            vector = (hyph2.end.pos(t0)-hyph.end.pos(t0))/np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))
            vertical_vector=np.array([-1,0])
            dot_product = np.dot(vertical_vector,vector)
            if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
                angle = np.arccos(dot_product)/(2*np.pi)*360
            else:
                angle = -np.arccos(dot_product)/(2*np.pi)*360
            score = np.cos((angle-(180+hyph.end.edges(t0)[0].orientation_begin(t0,30)))/360*2*np.pi)+np.cos((360+angle-hyph2.end.edges(t0)[0].orientation_begin(t0,30))/360*2*np.pi)
            if np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))<=500 and score>=0.5:
                hyph_anas_tip_tip.append((hyph,hyph2,t0))

In [13]:
len(hyph_anas_tip_hyph)

282

In [14]:
(len(hyph_anas_tip_tip)+len(hyph_anas_tip_hyph))

326

In [15]:
(len(hyph_anas_tip_tip)+len(hyph_anas_tip_hyph))/((len(hyph_anas_tip_tip)+len(hyph_anas_tip_hyph))+len(solved_nodes))

0.8190954773869347

In [16]:
(len(hyph_anas_tip_tip)+len(hyph_anas_tip_hyph))/len(exp_clean.hyphaes)

0.06770508826583593

In [84]:
plt.close('all')
node = choice(exp_clean.nodes)
t0 = choice(node.ts())
exp_clean.plot([t0],[[node.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[7, 8]

In [65]:
hyph

Hyphae(3272,3271)

In [76]:
plt.close('all')
hyph = choice(exp_clean.hyphaes)
t0 = choice(hyph.ts)
exp_clean.plot([t0,t0+1],[[hyph.end.label,hyph.root.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
plt.close('all')
hyph = choice(hyph_anas_tip_hyph)
t0 = get_time_anas_tip_hyph(hyph)
exp_clean.plot([t0,t0+1],[[hyph.end.label,hyph.root.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
plt.close('all')
hyph = choice(exp_clean.hyphaes)
exp_clean.plot([hyph.ts[-1]],[[hyph.end.label,hyph.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
hyph,hyph2,t0 = choice(hyph_anas_tip_tip)

In [29]:
exp_clean.plot([t0,t0+1],[[hyph.end.label,hyph.root.label,hyph2.end.label,hyph2.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
def get_time_anas_tip_hyph(anas_tip_hyph):
    for t in anas_tip_hyph.ts:
        if anas_tip_hyph.end.degree(t)>=3:
            return(t-1)
def get_time_anas_tip_tip(anas_tip_tip):
    return(anas_tip_tip.ts[-1])

In [10]:
anas_tip_hyph_observed = np.array([len([hyph for hyph in hyph_anas_tip_hyph if get_time_anas_tip_hyph(hyph)==t]) for t in range(len(exp_clean.nx_graph)-2)])

In [11]:
anas_tip_hyph_observed_lists = [[hyph.end.label for hyph in hyph_anas_tip_hyph if get_time_anas_tip_hyph(hyph)==t] for t in range(len(exp_clean.nx_graph)-2)]
anas_tip_tip_observed_lists = [[hyphs[0].end.label for hyphs in hyph_anas_tip_tip if hyphs[2]==t] for t in range(len(exp_clean.nx_graph)-2)]

In [ ]:
for t in range(len(anas_tip_hyph_observed_lists)):
    plt.close('all')
    exp_clean.plot([t],[anas_tip_hyph_observed_lists[t]+anas_tip_tip_observed_lists[t]],compress = 10,save=f'Data/anas/whole_net_{t}')

In [65]:
images = []
for t in range(len(anas_tip_hyph_observed_lists)):
    images.append(imageio.imread(f'Data/anas/whole_net_{t}.png'))
imageio.mimsave(f'Data/anas/whole_net.gif', images,duration=2)

In [12]:
anas_tip_tip_observed = np.array([len([hyphs for hyphs in hyph_anas_tip_tip if hyphs[2]==t])//2 for t in range(len(exp_clean.nx_graph)-2)])

In [27]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ts = np.array(range(len(exp_clean.nx_graph)-2))*4
ax.plot(ts,anas_tip_hyph_observed+anas_tip_tip_observed,label='spotted anastomosis')
ax.plot(ts,number_anas_theory, label = 'theoretical number of anastomosis')
ax.set_xlabel('time')
ax.set_ylabel('number of anastomosis')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
anastomosing_tips = {hyph for hyph in hyph_anas_tip_hyph}.union({c[0] for c in hyph_anas_tip_tip}.union({c[1] for c in hyph_anas_tip_tip}))

In [14]:
# hyph = choice(list(anastomosing_tips))
length_anas = [hyph.get_length_pixel(hyph.ts[-1]) for hyph in anastomosing_tips]

In [15]:
length_all = [hyph.get_length_pixel(hyph.ts[-1]) for hyph in exp_clean.hyphaes if len(hyph.ts)>=1]

In [18]:
length_all_real = [get_length(hyph,hyph.ts[-1]) for hyph in exp_clean.hyphaes if len(hyph.ts)>=1]

In [20]:
length_anas = [hyph.get_length_pixel(hyph.ts[-1]) for hyph in anastomosing_tips]

In [19]:
length_anas_real = [get_length(hyph,hyph.ts[-1]) for hyph in anastomosing_tips]

In [17]:
def get_length(hyph,t):
    pixel_conversion_factor = 1.725
    nodes,edges = hyph.get_nodes_within(t)
    length=0
    for edge in edges:
        length_edge = 0
        pixels = edge.pixel_list(t)
        for i in range(len(pixels)//10+1):
            if i*10<=len(pixels)-1:
                length_edge+=np.linalg.norm(np.array(pixels[i*10])-np.array(pixels[min((i+1)*10,len(pixels)-1)]))
#         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
        length+=length_edge
    return(length*pixel_conversion_factor)

In [21]:
pixel_conversion_factor = 1.725
(len(hyph_anas_tip_tip)+len(hyph_anas_tip_hyph))/(np.sum(length_all_real)/10000)

1.3210740796068454

In [26]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(np.log(length_all_real),60)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  1.,   0.,   6.,   0.,  10.,   0.,   7.,   6.,  15.,  18.,  16.,
         18.,   8.,  14.,  23.,  21.,  21.,  26.,  14.,  35.,  39.,  36.,
         45.,  61.,  64.,  72.,  73.,  82.,  97., 120., 107., 118., 114.,
        126., 101.,  87.,  73.,  72.,  60.,  44.,  43.,  38.,  43.,  28.,
         29.,  26.,  24.,  22.,  22.,  13.,  13.,  14.,   5.,   5.,   6.,
          6.,   1.,   2.,   2.,   2.]),
 array([ 0.54522705,  0.70950972,  0.87379239,  1.03807506,  1.20235773,
         1.3666404 ,  1.53092307,  1.69520574,  1.85948841,  2.02377108,
         2.18805375,  2.35233642,  2.51661909,  2.68090176,  2.84518443,
         3.0094671 ,  3.17374977,  3.33803244,  3.50231511,  3.66659778,
         3.83088045,  3.99516312,  4.15944579,  4.32372846,  4.48801113,
         4.6522938 ,  4.81657647,  4.98085914,  5.14514181,  5.30942448,
         5.47370715,  5.63798982,  5.80227249,  5.96655516,  6.13083783,
         6.2951205 ,  6.45940316,  6.62368583,  6.7879685 ,  6.95225117,
      

In [24]:
len(length_all_real)

2194

In [22]:
def plot_loghist(x, bins):
    hist, bins = np.histogram(x, bins=bins)
    logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    plt.close('all')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.hist(x, bins=logbins,color='orange')
    ax.set_xscale('log')
    ax.set_xlabel('total length')
    ax.set_ylabel('number of hyphaes')

In [23]:
plot_loghist(length_all_real,60)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
from hyphae_id import get_pixel_growth_and_new_children

In [122]:
growing_hyphae = [[hyph for hyph in exp_clean.hyphaes if t in hyph.ts and (t+1) in hyph.ts and len([element for liste in get_pixel_growth_and_new_children(hyph,t,t+1)[0] for element in liste])>=30] for t in range(len(exp_clean.nx_graph)-1)]

In [123]:
growing_and_stopping = []
for hyph in exp_clean.hyphaes:
    for t in range((len(exp_clean.nx_graph)-2)):
        if hyph in growing_hyphae[t] and hyph not in growing_hyphae[-1]:
            growing_and_stopping.append((hyph,t))
            break

In [85]:
def get_length_real(pixel_list):
    pixels = pixel_list
    length=0
    pixel_conversion_factor = 1.725
    for i in range(len(pixels)//10+1):
        if i*10<=len(pixels)-1:
            length+=np.linalg.norm(np.array(pixels[i*10])-np.array(pixels[min((i+1)*10,len(pixels)-1)]))
#         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
    return(length*pixel_conversion_factor)

In [128]:
plt.close('all')
hypha,t0 = choice(growing_and_stopping)

In [133]:
growth_dic = {}
for hypha,t0 in growing_and_stopping:
    growths=[]
    for t in range(t0,(len(exp_clean.nx_graph)-2)):
        if t in hypha.ts and (t+1) in hypha.ts:
            growth=get_length_real([element for liste in get_pixel_growth_and_new_children(hypha,t,t+1)[0] for element in liste])
            growths.append(growth)
    growth_dic[hypha]=growths

In [130]:
for 

(Hyphae(4137,4138),
 [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])

In [131]:
exp_clean.plot([20,21,22],[[hypha.end.label,hypha.root.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
exp_clean.plot([t,t+1,t+2],[[hyph.end.label,hyph.root.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')
for t in ts
exp_clean.plot([])

In [43]:
np.mean(length_anas),np.std(length_anas)

(1114.7932330827068, 1632.2649995255101)

In [44]:
np.mean(length_all),np.std(length_all)

(478.0355515041021, 1157.7276089836505)